# DPA介绍以及汉明设想

## DPA攻击理论

* 我们假设数据线和测量的功率之间存在关系，类似下图

    ![](img/dpa_4bits_powerhw_scaled.png)

    如何证明它呢？我们可以画出能量轨迹的汉明重量图像。

## 轨迹分析

* 在记录的能轨中，`trace_array`是轨迹数据，`textin_array`是加密的明文。现在我们来获取一些基本信息以便之后使用

In [ ]:
numtraces = np.shape(trace_array)[0] #轨迹总数
numpoints = np.shape(trace_array)[1] #每个轨迹的记录数

## 计算汉明重量

* 回顾AES工作流程，

    ![](img/Sbox_cpa_detail.png)

*（Xenny）我们的攻击点则在`SubBytes`也就是S盒置换之后，之前在测量信噪比一章中已经提到了这点。

* 所以我们可以编写一个函数计算一个输入字节，它的输出是

In [ ]:
def intermediate(pt, keyguess):
    return sbox[pt ^ keyguess]

* 所以最终我们得到了S盒输出的汉明重量，这里我们假设系统泄漏出了这里的汉明重量。

* （Xenny）然后教程中以第一位的HW值给不同的轨迹标上了颜色

## 找到平均值

* 要在这些汉明重量值和电压中找到“最好的地方”我们可以编写一个循环去检查每一个点的相关性并且找到最大值。在这么多点中我们只需要选取一个[500,1000]左右区间即可。下面展示了如何从足够的点中找到想要的相关值，通常我们不关心相关值的正负性而是直接使用绝对值去寻找最大值。
* （Xenny）这里的“最好的地方”指的是上一步绘制HW中选取的特殊区域（some specific area），以便在后面更好的进行攻击。

In [ ]:
import numpy as np

max_corr = 0
point = 0
for avg_point in range(500, 1400):
    hw_list = [[], [], [], [], [], [], [], [], []]
    for trace in traces:
        hw_of_byte = HW[intermediate(trace.textin[bnum], trace.key[bnum])]
        hw_list[hw_of_byte].append(trace.wave[avg_point])
    
    hw_mean_list = [np.mean(hw_list[i]) for i in range(0, 9)]
    
    corr = abs(np.corrcoef(range(1,9), hw_mean_list[1:9])[0,1])
    if corr > max_corr:
        max_corr = corr
        point = avg_point
        
print(max_corr, point)

* 其实如果倒回去看之前的图片的话，你也会发现几个高相关性点拥有不同的颜色

    我们可以考虑他们是线性(指HW值和电压之间)的，下面我们选取一个点可以来看看平均值的情况。

    ![](img/bokeh_plot.png)

    那么到这里其实我们已经看到了HW和电压的相关性了，当然我们发现斜率不符合预期，我们最开始的预期是HW值越大，电压越大，而这里则是相反，那是因为和我们的测量方式有关

    ![](img/vmeasure.png)

    如果所示，因为是串联电路，我们测量点的电压随着电流增加而减少。